###### Progetto di Simulazione Numerica
###### Professore Andrea Vicerè
###### Studente: Gabriele Ramagnano, n°matricola = 315439

# Tunnelling in a quantum system

The assigned task consists in simulating the trend of a wave function $\psi(\mathbf{x}, t)$ which describes the state of a particle in the quantum world. The quantity $\psi(\mathbf{x}, t)$ is actually a *complex* function, having both a real and an imaginary part.

The calculation of the time evolution of the function $\psi(\mathbf{x}, t)$ is made possible thanks to the use of the Schrödinger equation

\begin{equation}
i\hbar\frac{\partial \psi}{\partial t}=-\frac{\hbar^2}{2 M}\nabla^2 \psi + V(\mathbf{x}) \psi
\end{equation}

Simplifying the Schrödinger equation from two dimensions to one dimension

\begin{equation}
i\hbar\frac{\partial \psi}{\partial t}=-\frac{\hbar^2}{2 M}\frac{\partial^2 \psi}{d x^2}
\end{equation}

we obtain an equation very similar to the diffusion equation (Jupyter Notebook 03_03_1DDiffusion), in which there is a first order derivative in time, a second order derivative in space and a constant.

\begin{equation}
\frac{\partial u}{\partial t}= \nu \frac{\partial^2 u}{\partial x^2}
\end{equation}

Therefore the only novelty is represented by the presence of complex numbers which will then have to be managed during the course of the simulation.


In [1]:
import sys
import time as clock

class ProgressBar:
    
    def __init__(self, total_iterations):
        
        self.total_iterations = total_iterations
        self.current_iteration = 0
        self.start_time = clock.time()
        
    def step(self):
        
        self.current_iteration += 1
        self.show_progress()
        if self.current_iteration == self.total_iterations:
            self.end_time = clock.time()
            print(f'\Total time taken: {self.end_time -self.start_time} seconds')
    
    def show_progress(self):
        
        percent_complete = self.current_iteration / self.total_iterations
        num_bars = int(percent_complete * 20)
        sys.stdout.write('\r')
        sys.stdout.write(f"Progress: [{'=' * num_bars}{' ' * (20 - num_bars)}]{int(percent_complete * 100)}%")
        sys.stdout.flush()
    
        

In [2]:
import numpy
import sympy
from sympy import init_printing
init_printing()

from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

#animation
import matplotlib
from matplotlib import animation
from IPython.display import HTML

from sympy.utilities.lambdify import lambdify

## Assignment 0: write down code

Write down a *structured* code (or a class) for simulating the time evolution of the waveform over time. The code should be written according to the following general specifications:
* the state of the waveform should be represented by the values of the complex amplitude $\psi$ defined using a complex `numpy` array. The size of the array should be a parameter for any function using the array. If using a class, clearly $\Delta x$ could be a useful class element.
* A method should allow setting the initial values in the array by means of a defined function, and the $\Delta x$ value chosen for the discretization.
* A method should allow evolving by one step the values of $\psi$, taking as parameters the time step $\Delta t$, the discretization step $\Delta x$, the mass $M$ and a function $V(x)$ representing the potential.
* A method should allow computing the probability distribution $P(x,t)$.
* A method should allow plotting the probability distribution, and the real and imaginary parts of $\psi$.
* A method should allow integrating $P(x,t)$ over $d x$, for instance to check the normalization. 

Use zero boundary conditions: that is $\psi=0$ at the boundary.

In [3]:
class QuantumTunnellingSimulator:
    
    def __init__(self, time, Delta_x, Delta_t ):
       
        self.e_mass       = 1e-30                          # mass of the electron
        self.vg           = 1e-1                           # group speed (m/s)
        self.momentum     = self.vg * self.e_mass          # momentum (kg * m/s) 
        self.plank_redux  = 1.054e-34                      # reduced Planck constant (J * s)
        self.width0       = 2e-2                           # initial dispersion (m)

        self.T            = time                           # simulation time (s)
        self.dx           = Delta_x                        # space step (m)
        self.dt           = Delta_t                        # time step
        self.x_0          = 1e-1                           # starting position (m)
        self.x_min        = 0                              # initial position (m)
        self.x_max        = 1                              # final position (m)
        self.nx           = int(self.x_max / self.dx) + 1  # number of grid points
        self.nt           = int(self.T / self.dt)          # total time steps
        
        self.x_values     = numpy.linspace(self.x_min, self.x_max , self.nx) 
        
        self.K            = (self.momentum**2) / (2 * self.e_mass)  # kinetic energy of the particles
        self.v0           = numpy.array([0 * self.K, -2 * self.K, -1 * self.K, -1/2 * self.K,
                                         1 * self.K, 1/2 * self.K, 2 * self.K])
        
        self.psi          = None
        self.real_psi     = None
        self.imag_psi     = None
        self.probab_distr = None 
      
    # FUNCTIONS REQUIRED BY: ASSIGNMENT_0: WRITE DOWN CODE
    
    # method to set the initial values of wavefunction
    def initialize_wavefunction(self, x, x0, sigma0, p0, h):
        
        wave_packet = sympy.exp(-((x-x0)**2 / (4 * sigma0**2))) / sympy.sqrt(sympy.sqrt(2 * sympy.pi) * sigma0) 
        exponential = sympy.exp((p0 * 1j * (x-x0)/ h))
        wave        = wave_packet * exponential
        wave_lamb   = lambdify((x0, x, p0, sigma0, h), wave)
        
        self.psi    = numpy.asarray([wave_lamb(self.x_0, x, self.momentum, self.width0, self.plank_redux) 
                                     for x in self.x_values])
        
    # method to evolving by one step the values of psi
    def evolve_one_step(self, dt, dx, v):
    
        un       = numpy.empty(self.nx)
        en       = numpy.empty(self.nx)
    
        self.real_psi = numpy.real(self.psi)
        self.imag_psi = numpy.imag(self.psi)
        
        #real part
        en = self.imag_psi.copy()
        self.real_psi[1:-1] +=  - (self.plank_redux / (2 * self.e_mass)) * dt / \
                               dx**2*(en[2:] - 2*en[1:-1] + en[0:-2]) \
                               + (dt / self.plank_redux) * v[1:-1] * en[1:-1]
        #At the boundary
        self.real_psi[0]    = 0
        self.real_psi[-1]   = 0
    
        #imaginary part
        un = self.real_psi.copy()   
        self.imag_psi[1:-1] +=  (self.plank_redux / (2 * self.e_mass)) * dt / \
                                dx**2*(un[2:] - 2*un[1:-1] + un[0:-2]) \
                                - (dt / self.plank_redux) * v[1:-1] * un[1:-1]
        #At the boundary    
        self.imag_psi[0]    = 0
        self.imag_psi[-1]   = 0  
    
    # method to calculate probability distribution
    def probability_distribution(self):
        self.psi = self.real_psi + 1j * self.imag_psi
        self.probab_distr = abs(self.psi)**2
        
    # method to plotting probability distribution, and the real and imaginary part of psi
    def plot_wavefunction(self):
        pyplot.plot(self.x_values, self.probab_distr, color='#003366', ls='-', lw=3)
        pyplot.plot(self.x_values, self.real_psi, color='#003366', ls='-', lw=3)
        pyplot.plot(self.x_values, self.imag_psi, color='#003366', ls='-', lw=3)
    
    # method to integrating probability distribution over dx       
    def integrate_probability_distribution(self):
        return numpy.sum(self.probab_distr) * (self.x_values[1] - self.x_values[0])
        
    
    #---------------------------------------------------------------------------additional methods
    
    # methode to initialize wavefunction with modified amplitude
    def initialize_wavefunction_amplitude(self, x, x0, sigma0, p0, h, t, M, amplitude):
        
        wave_packet = sympy.exp(-((x-x0)**2 / (4 * sigma0**2))) / sympy.sqrt(sympy.sqrt(2 * sympy.pi) * sigma0) 
        exponential = sympy.exp((p0 * 1j * (x-x0)/ h))
        wave        = amplitude * wave_packet * exponential
        wave_lamb   = lambdify((x0, x, p0, sigma0, h), wave)
        
        self.psi    = numpy.asarray([wave_lamb(self.x_0, x, self.momentum, self.width0, self.plank_redux) 
                                     for x in self.x_values])
    
    # method to initialize the exact probability of distribution
    def initialize_distribution_exact(self, x, x0, sigma0, p0, h, t, M):
        return sigma0 / sympy.sqrt(2 * sympy.pi) * sympy.exp(-(sigma0**2 * (x - ((p0 * t) / M))**2) / \
               (2 * ( sigma0**4 + (t**2 * h**2) / (4 * M**2) ))) / sympy.sqrt(sigma0**4 + (t**2 * h**2) / (4 * M**2) ) 
    
    # method to simulate the exact probability of distribution
    def simulate_distribuition_exact(self, distribution_exact, x, x0, sigma0, p0, h, dt, t , M, num_steps):
        distrib_ex_lamb = lambdify((x0, x, p0, sigma0, h, t , M), distribution_exact)
        return numpy.asarray([distrib_ex_lamb(self.x_0, x, self.momentum, self.width0, self.plank_redux,
                              num_steps * dt, self.e_mass) for x in self.x_values])
    
    # method to plotting probability of distribution
    def plot_distribuition_exact(self, distribution_exact):
        pyplot.plot(self.x_values, distribution_exact, color='r', ls='--', lw=2)
    
    def plot_psi(self):
        pyplot.plot(self.x_values, abs(self.psi)**2, color='#003366',  ls='-', lw=2)
    
    # method to setting potential energy 
    def set_potential(self, num_potential):
        v = numpy.ones(self.nx) 
        v[numpy.intersect1d(numpy.where(self.x_values >= 5.0 * 10**-1), 
                            numpy.where(self.x_values <= 5.5 * 10**-1))] = self.v0[num_potential]
    
        v[numpy.where( self.x_values < 5.0 * 10**-1 )] = 0
        v[numpy.where( self.x_values > 5.5 * 10**-1 )] = 0
        
        return v  
    
    # method to plotting potential energy
    def plot_potential(self, potential):
        pyplot.plot(self.x_values, potential, color='#003366', ls='-', lw=2)
    
    # set time simulation
    def set_time(self, time):
        self.T = time
        
    # set number of steps 
    def set_num_steps(self, steps):
        self.nt = steps
        

In [ ]:
time      = 0                    
Delta_x   = 1e-4                  
Delta_t   = 9e-5   
num_steps = int(time / Delta_t)  
x, x0, sigma0, p0, h, t, M = sympy.symbols('x x0 sigma0 p0 h t M')

# let's build the QuantumTunnellingSimulator object
simulator = QuantumTunnellingSimulator(time, Delta_x, Delta_t)

## Exact solution for $V=0$

By reducing the Schrödinger equation

\begin{equation}
i\hbar\frac{\partial \psi}{\partial t}=-\frac{\hbar^2}{2 M}\frac{\partial^2 \psi}{d x^2}
\end{equation}

it can be shown that the solution takes the following form:

\begin{equation}
\psi(x, t)=\sqrt{\frac{\sigma_0}{\sqrt{2\pi}}}\left[\frac{\exp \left(-\frac{\left(x-\frac{p_0 t}{M}\right)^2}{4 \left(\sigma_0^2+\frac{i t \hbar }{2 M}\right)}\right)}{\sqrt{\left(\sigma_0^2+\frac{i t \hbar }{2 M}\right)}}\right]\exp\left(\frac{i \left(p_0 x-\frac{p_0^2 t}{2 M}\right)}{\hbar}\right)
\end{equation}


However, since we are directly interested in drawing the square modulus of the wave, we use this modulus directly here

\begin{equation}
P(x,t)=\left|\psi(x,t)\right|^2=\frac{\sigma_0}{\sqrt{2\pi}}\frac{\exp \left(-\frac{ \sigma_0^2
    (x -\frac{p_0 t}{M})^2}{2 \left(\sigma_0^4 +\frac{t^2 \hbar^2}{4 M^2}\right)}\right) }{\sqrt{\sigma_0^4+\frac{t^2 \hbar^2}{4 M^2}}}
\end{equation}

which represents the probability of a particle being at position $x$.


In [ ]:
# a) visualize the analytical solution at time t = 0
analytic_distribution = simulator.initialize_distribution_exact(x, x0, sigma0, p0, h, t, M)
distribution_exact    = simulator.simulate_distribuition_exact(analytic_distribution, x, x0, sigma0, p0, h, Delta_t, t , M, int((time + 1)/ Delta_t))
simulator.plot_distribuition_exact(distribution_exact)

### Initial and Boundary Conditions

The initial conditions of the function for the case $V=0$ are given by the following analytical solution:

\begin{equation}
\psi(x, t)=\left[\frac{\exp \left(-\frac{(x-x_0)^2}{4 \sigma_0^2}\right)}{\sqrt[4]{2\pi}\sqrt{\sigma_0}}\right]\exp\left(\frac{i\,p_0 \left(x-x_0\right)}{\hbar}\right)
\end{equation}

we use zero boundary conditions: that is $\psi=0$ at the boundary.

In [ ]:
# b) display the numerical solution at time t = 0

simulator.initialize_wavefunction(x, x0, sigma0, p0, h)
simulator.plot_psi()

In [ ]:
# let's see them both
simulator.plot_distribuition_exact(distribution_exact)
simulator.plot_psi()

In [ ]:
# c) let us compare the numerical and analytical solution at time t = 8 in a single graph 

# let's set the initial values
time      = 8
num_steps = int(time / Delta_t)
num_v0    = 0
simulator.set_time(time)
simulator.set_num_steps(num_steps)
potential = simulator.set_potential(num_v0)

## Simulating in time

We will try to simulate the evolution of the waveform using a simple forward difference in time, and a central scheme in space: in other words, we will introduce the following two approximations

\begin{eqnarray}
\frac{\partial\psi(x,t)}{\partial t} &\simeq& \frac{\psi(x,t+\Delta t) - \psi(x,t)}{\Delta t}\\
\frac{\partial^2\psi(x,t)}{\partial x^2} &\simeq& \frac{\psi(x+\Delta x,t) - 2 \psi(x,t) + \psi(x-\Delta x, t)}{\left(\Delta x\right)^2}
\end{eqnarray}

Now, we write two separate, coupled equations for the real and imaginary parts

\begin{equation}
\psi(x,t) = \psi_R(x,t) + i \psi_I(x,t)
\end{equation}

hence

\begin{eqnarray}
\hbar \frac{\partial\psi_R(x,t)}{\partial t} &=& -\frac{\hbar^2}{2 M}\frac{\partial^2\psi_I(x,t)}{\partial x^2} + V(x)\psi_I(x,t)\\
\hbar \frac{\partial\psi_I(x,t)}{\partial t} &=& +\frac{\hbar^2}{2 M}\frac{\partial^2\psi_R(x,t)}{\partial x^2} - V(x)\psi_R(x,t)\ .
\end{eqnarray}

Let's take a "step back". Let's start again from the Schrödinger equation.

\begin{equation}
i\hbar\frac{\partial \psi}{\partial t}=-\frac{\hbar^2}{2 M}\frac{\partial^2 \psi}{d x^2}
\end{equation}

To get the next time step, we solve the given differential equation using the finite difference method explicitly. So be it ($V(\mathbf{x}) = 0$):

\begin{equation}
i\hbar\frac{\psi_{i}^{n+1}-\psi_{i}^{n}}{\Delta t}=-\frac{\hbar^2}{2 M}\frac{\psi_{i+1}^{n}-2\psi_{i}^{n}+\psi_{i-1}^{n}}{\Delta x^2}
\end{equation}

To get the next time step $\psi_{i}^{n+1}$, we can solve the equation as follows:

\begin{equation}
\psi_{i}^{n+1} = \psi_{i}^{n}-\frac{i\hbar\Delta t}{2M\Delta x^2}(\psi_{i+1}^{n}-2\psi_{i}^{n}+\psi_{i-1}^{n})
\end{equation}


Now let's rewrite the real and imaginary part of $\psi$ following the same reasoning:

\begin{equation}
\psi_{R,i}^{n+1} = \psi_{R,i}^{n}-\frac{\hbar\Delta t}{2M\Delta x^2}(\psi_{I,i+1}^{n}-2\psi_{I,i}^{n}+\psi_{I,i-1}^{n})
\\
\psi_{I,i}^{n+1} = \psi_{I,i}^{n}+\frac{\hbar\Delta t}{2M\Delta x^2}(\psi_{R,i+1}^{n}-2\psi_{R,i}^{n}+\psi_{R,i-1}^{n})
\end{equation}


To make the implementation more efficient we can manipulate the system of equations like this:

\begin{equation}
\psi_{R,i}^{n+1} = \psi_{R,i}^{n}-\frac{\hbar\Delta t}{2M\Delta x^2}(\psi_{I,i}^{n + 1}-2\psi_{I,i}^{n}+\psi_{I,i-1}^{n})
\\
\psi_{I,i}^{n+1} = \psi_{I,i}^{n}+\frac{\hbar\Delta t}{2M\Delta x^2}(\psi_{R,i+1}^{n + 1}-2\psi_{R,i}^{n + 1}+\psi_{R,i-1}^{n + 1})
\end{equation}


In [ ]:
analytic_distribution = simulator.initialize_distribution_exact(x, x0, sigma0, p0, h, t, M)
distribution_exact    = simulator.simulate_distribuition_exact(analytic_distribution, x, x0, sigma0, p0, h, Delta_t, t , M, int((time + 1)/ Delta_t))
simulator.initialize_wavefunction(x, x0, sigma0, p0, h)

pb = ProgressBar(num_steps)
for n in range(num_steps):
    simulator.evolve_one_step(Delta_t, Delta_x, potential)
    pb.step()

In [ ]:
simulator.plot_distribuition_exact(distribution_exact)
simulator.plot_psi()

# Be ware CFL

In choosing the time step $\Delta t$ we have considered a number lower than 1e-4, ie the size of the space step $\Delta x$. In this way the stability of the simulation is guaranteed and there is no risk that the numerical solution will explode! However, a size of the time step that is not too "narrow" has been chosen so as not to increase the computation cycles.

In [ ]:
# d) simulate the numerical solution up to t = 8 (v_0 = 0)
simulator.initialize_wavefunction(x, x0, sigma0, p0, h)

In [ ]:
fig = pyplot.figure(figsize=(8,5))
ax = pyplot.axes(xlim=(0,1), ylim=(-6,20))
line  = ax.plot([], [], color='r', ls='-', lw=3, label='square module of psi')[0]
line1 = ax.plot([], [], color='#008000', ls='-', lw=1, label='real psi')[0]
line2 = ax.plot([], [], color='#FFA500', ls='-', lw=0.5, label='imaginary psi')[0]
pyplot.legend(fontsize= 7);

In [ ]:
steps_cutter = 1111
def wave(n):
    
    pb = ProgressBar(steps_cutter)
    for n in range(steps_cutter):
        simulator.evolve_one_step(Delta_t, Delta_x, potential)
        pb.step()
    
    p = simulator.real_psi + 1j * simulator.imag_psi
    line.set_data(simulator.x_values, abs(simulator.psi)**2)
    line1.set_data(simulator.x_values, simulator.real_psi)
    line2.set_data(simulator.x_values, simulator.imag_psi)
    

In [ ]:
anim = animation.FuncAnimation(fig, wave,
                               frames=80, interval=100)

In [ ]:
HTML(anim.to_jshtml())

# Check convergence

To compare solutions, we will use the distance between them normalized by the norm of the exact solution, as follows

\begin{equation}
error = \frac{||{\Psi - \Psi_{exact}}||_2}{||\Psi_{exact}||_2}
\end{equation}

In [ ]:
# e) view the graph for L2_norm

def L2_error(Psi, Psi_exact):
    """Computes L2 norm of error
    
    Parameters:
    ----------
    Psi      : array of complex
        array with numerical solution
    Psi_exact: array of complex
        array with exact solution
    Returns:
    -------
    error: L2 norm of error
    """
    
    error = numpy.sqrt(numpy.sum((Psi-Psi_exact)**2)/numpy.sum(Psi_exact**2))
    
    return abs(error)

In [ ]:
def check_convergence(T, dt_values, nt_values, psi_ex, V, x, x0, sigma0, p0, h, t, M):

    dt_values = dt_values
    nt_values = nt_values
    
    error = numpy.zeros(len(dt_values))
    
    for i, dt in enumerate(dt_values):
        
        # Let us set the initial conditions of the numerical solution
        simulator.initialize_wavefunction(x, x0, sigma0, p0, h)
    
        # we compute the numerical solution
        pb = ProgressBar(int(nt_values[i]))
        for n in range(int(nt_values[i])):
            simulator.evolve_one_step(dt, Delta_x, V)
            pb.step()
    
        # we compute the analytical solution
        Psi_exact = simulator.simulate_distribuition_exact(psi_ex, x, x0, sigma0, p0, h, dt, 
                                                           t , M, ( (T + 1) / dt) + 1)
        
        error[i] = L2_error(simulator.psi, Psi_exact)
        
    return error

In [ ]:
time      = 1                    
num_steps = int(time / Delta_t)

simulator.set_time(time)                      # we reset the simulation time
simulator.set_num_steps(num_steps)            # we reset the number of total time steps

num_v0    = 0
potential = simulator.set_potential(num_v0)
dt_values = numpy.asarray([9e-5, 8e-5, 7e-5, 9e-6])    
nt_values = time / dt_values

# let's initialize the distribution function
distribution = simulator.initialize_distribution_exact(x, x0, sigma0, p0, h, t, M) 

# let's initialize the wave function to be simulated
simulator.initialize_wavefunction(x, x0, sigma0, p0, h)

# let's check the convergence...
error = check_convergence(time, dt_values, nt_values, distribution, potential, x, x0, sigma0, p0, h, t, M)

In [ ]:
pyplot.figure(figsize=(12,12))
pyplot.grid(True)
pyplot.xlabel(r'$\Delta t$', fontsize=18)
pyplot.ylabel(r'$L_2$-norm of the error', fontsize=18)
pyplot.loglog(nt_values, error, color='k', ls='--', lw=2, marker='o')
pyplot.legend(['FTCS']);

## Assignment 2: use the code for $V\neq 0$

Now that we are confident that our code is not wrong, we carry out the same simulation as before, but this time introducing a non-zero potential, which has the following shape

\begin{eqnarray}
V(x) = 0\quad &\textrm{for}& x \in [0.0, 5.0\times 10^{-1} m)\\
V(x) = V_0\quad &\textrm{for}& x \in [5.0\times 10^{-1} m, 5.5\times 10^{-1} m]\\
V(x) = 0\quad &\textrm{for}& x \in (5.5\times 10^{-1} m, 1 m]
\end{eqnarray}

To choose the value of $V_0$, let us point out that the kinetic energy of our particle, which is coming from the left side, is

\begin{equation}
E_0\sim \frac{p_0^2}{2 M} \simeq 5\times 10^{-33} J 
\end{equation}

Now we want to see what happens for different values of $V_0$, say for
1. $V_0 = - 2\,E_0$
2. $V_0 = - E_0$
3. $V_0 = - \frac{1}{2} E_0$
4. $V_0 = + E_0$
5. $V_0 = + \frac{1}{2} E_0$
6. $V_0 = + 2\,E_0$

In [ ]:
# f) view the wave function plot for all types of potential barriers

# let's set the initial
time      = 8
num_steps = int(time / Delta_t)
num_v0    = 1
simulator.set_time(time)
simulator.set_num_steps(num_steps)
potential = simulator.set_potential(num_v0)

In [ ]:
amplitude = 2.1e-17
simulator.initialize_wavefunction_amplitude(x, x0, sigma0, p0, h, t, M, amplitude)

In [ ]:
fig = pyplot.figure(figsize=(8,5))
simulator.plot_potential
ax = pyplot.axes(xlim=(0,1), ylim=(-1.5e-32,2e-32))
pyplot.plot(simulator.x_values, potential, color='r', ls='-', lw=2)
line  = ax.plot([], [], color='#003366', ls='-', lw=3)[0]

In [ ]:
slice_steps = 1111
def wave(n):
    
    pb = ProgressBar(slice_steps)
    for n in range(slice_steps):
        simulator.evolve_one_step(Delta_t, Delta_x, potential)
        pb.step()
    
    p = simulator.real_psi + 1j * simulator.imag_psi
    line.set_data(simulator.x_values, abs(simulator.psi)**2)

In [ ]:
anim = animation.FuncAnimation(fig, wave,
                               frames=80, interval=100)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
def wave_animation(time, frames, slice_steps, num_v0, amplitude):
    # let's set the initial
    time      = time
    Delta_t   = 9e-5  
    num_steps = int(time / Delta_t)
    num_v0    = num_v0
    simulator.set_time(time)
    simulator.set_num_steps(num_steps)
    potential = simulator.set_potential(num_v0)
    
    simulator.initialize_wavefunction_amplitude(x, x0, sigma0, p0, h, t, M, amplitude)
    
    fig = pyplot.figure(figsize=(8,5))
    simulator.plot_potential
    ax = pyplot.axes(xlim=(0,1), ylim=(-1.5e-32,2e-32))
    pyplot.plot(simulator.x_values, potential, color='r', ls='-', lw=2)
    line  = ax.plot([], [], color='#003366', ls='-', lw=3)[0]
    
    slice_steps = slice_steps
    def wave(n):
    
        pb = ProgressBar(slice_steps)
        for n in range(slice_steps):
            simulator.evolve_one_step(Delta_t, Delta_x, potential)
            pb.step()
    
        p = simulator.real_psi + 1j * simulator.imag_psi
        line.set_data(simulator.x_values, abs(simulator.psi)**2)
    
    return animation.FuncAnimation(fig, wave,
                                   frames=frames, interval=100)

In [ ]:
HTML(wave_animation(8, 80, 1111, 0, 2.1e-17).to_jshtml())

In [ ]:
HTML(wave_animation(8, 80, 1111, 1, 2.1e-17).to_jshtml())

In [ ]:
HTML(wave_animation(8, 80, 1111, 2, 2.1e-17).to_jshtml())

In [ ]:
HTML(wave_animation(8, 80, 1111, 3, 2.1e-17).to_jshtml())

In [ ]:
HTML(wave_animation(8, 80, 1111, 4, 2.1e-17).to_jshtml())

In [ ]:
HTML(wave_animation(8, 80, 1111, 5, 2.1e-17).to_jshtml())

In [ ]:
HTML(wave_animation(8, 80, 1111, 6, 2.1e-17).to_jshtml())

# Integral of square modulus

Since the particle must be somewhere, for $P$ to be a true probability it is necessary that its integral over the whole space is $1$: for instance, in 1D one should have

\begin{equation}
\int_{-\infty}^{+\infty} P(x, t) d x = 1
\end{equation}

In [ ]:
# g) integral of the square modulus

# let's set the initial values
time      = 8
num_steps = int(time / Delta_t)
num_v0    = 0
simulator.set_time(time)
simulator.set_num_steps(num_steps)
potential = simulator.set_potential(num_v0)

In [ ]:
simulator.initialize_wavefunction(x, x0, sigma0, p0, h)

pb = ProgressBar(num_steps)
for n in range(num_steps):
    simulator.evolve_one_step(Delta_t, Delta_x, potential)
    pb.step()

In [ ]:
simulator.probability_distribution()
integrale = simulator.integrate_probability_distribution()
print("The integral of the square modulus of the function is: ", integrale)